
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## RBFs reconstruction of historical and synthetic data

inputs required: 
  * Synthetic offshore waves - emulator output
  * Sea and swell **SWAN simulated cases**

in this notebook:
  * RBF reconstruction simulated storms
  * Generation of hourly nearshore waves with Intradaily Hydrographs



### Workflow:

<div>
<img src="../resources/nb02_03.png" width="400px">
</div>



In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..', '..'))

# teslakit
from teslakit.database import Database, hyswan_db
from teslakit.rbf import RBF_Reconstruction, RBF_Validation
from teslakit.waves import Aggregate_WavesFamilies, AWL, Intradaily_Hydrograph, Snell_Propagation
from teslakit.climate_emulator import Climate_Emulator
from teslakit.util.time_operations import repair_times_hourly, add_max_storms_mask



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database
p_data = r'/media/administrador/HD/Dropbox/Guam/teslakit/data'
# p_data=r'/Users/laurac/Dropbox/Guam/teslakit/data'
# offshore
db = Database(p_data)
db.SetSite('GUAM')

# climate change - S5
db_S5 = Database(p_data)
db_S5.SetSite('GUAM_CC_S5')

# hyswan simulation database
db_sim = hyswan_db(db.paths.site.HYSWAN.sim)
db_sim_S5 = hyswan_db(db_S5.paths.site.HYSWAN.sim)


## Define profile to run

In [3]:
prf=9

In [4]:
# --------------------------------------
# aux.

def fix_target_nans(subset, target):
    'remove NaN data indexes from subset and target. RBF does not handle NaNs'
    
    r_nan = target.isnull().any(axis=1)  # find any row with nans

    if r_nan.any():
        # log
        print('remove nan data found at target:')
        print(target[r_nan])
        
        # clean data
        target = target[~r_nan]
        subset = subset[~r_nan]
        
    return subset, target


# Snell propagation 400m -> 21m
def snell_prop_data(waves_dset, d1, d2, ob):
    'teslakit/waves.py Snell_Propagation func. parser'
    
    # get variables
    Tp = waves_dset['TPsmoo'].values[:]
    Hs = waves_dset['Hsig'].values[:]
    Dir = waves_dset['Dir'].values[:]
  
    # TODO: check dir modifications and refactor
    dir_I = Dir.copy()

    # El angulo definido entre -90 y 90
    dir_I[np.where(dir_I > 180)[0]] = dir_I[np.where(dir_I > 180)[0]] - 360

    # Obligamos que el angulo este en este sector
    dir_I[np.where(dir_I > 90)[0]] = 85
    dir_I[np.where(dir_I <= -90)[0]] = -85

    Hs_p, Dir_p, ks, kp = Snell_Propagation(Tp, Hs, dir_I, d1, d2, ob)
    
    # return copy of input
    out = waves_dset.copy()
    out['Hsig'] = Hs_p
    out['Dir'] = Dir_p
    
    return out



## SIMULATION Waves Reconstruction

In [5]:
# --------------------------------------
# RBF Reconstruction data

# historical sea
wvs_sea_dataset = db_sim.Load('sea_dataset').loc[:,['hs','tp','dir']] 
wvs_sea_subset = db_sim.Load('sea_subset').loc[:,['hs','tp','dir']] 
wvs_sea_target = db_sim.Load('sea_target').reset_index() 

# historical swells
wvs_swl_dataset = db_sim.Load('swl_dataset').loc[:,['hs','tp','dir']]  
wvs_swl_subset = db_sim.Load('swl_subset').loc[:,['hs','tp','dir']] 
wvs_swl_target = db_sim.Load('swl_target').reset_index() 

wvs_swl_target=wvs_swl_target.loc[wvs_swl_target['profile'] ==prf].reset_index().loc[:,['Hsig','TPsmoo','Dir']]
wvs_sea_target=wvs_sea_target.loc[wvs_sea_target['profile'] ==prf].reset_index().loc[:,['Hsig','TPsmoo','Dir']]


In [6]:
# --------------------------------------
# prepare dataset, subset and target data (propagated waves)

# solve Hs coefficient
wvs_sea_target['Hsig'] = wvs_sea_target['Hsig']
wvs_swl_target['Hsig'] = wvs_swl_target['Hsig'] * wvs_swl_subset['hs']

# Snell propagation 
# depth_1, depth_2, bathy_dir = 400, 21, 0
# wvs_sea_target = snell_prop_data(wvs_sea_target, depth_1, depth_2, bathy_dir)
# wvs_swl_target = snell_prop_data(wvs_swl_target, depth_1, depth_2, bathy_dir)


# remove nans (if any) from subset and target
wvs_sea_subset, wvs_sea_target = fix_target_nans(wvs_sea_subset, wvs_sea_target)
wvs_swl_subset, wvs_swl_target = fix_target_nans(wvs_swl_subset, wvs_swl_target)


# ensure dataset and subset have same variables and column order
vns_ds = ['hs', 'tp', 'dir']

wvs_sea_dataset = wvs_sea_dataset[vns_ds].values
wvs_sea_subset = wvs_sea_subset[vns_ds].values

wvs_swl_dataset = wvs_swl_dataset[vns_ds].values
wvs_swl_subset = wvs_swl_subset[vns_ds].values


# select target variables
vns_tgt = ['Hsig', 'TPsmoo', 'Dir']

wvs_sea_target = wvs_sea_target[vns_tgt].values
wvs_swl_target = wvs_swl_target[vns_tgt].values


remove nan data found at target:
     Hsig  TPsmoo  Dir
5     0.0     NaN  NaN
22    0.0     NaN  NaN
33    0.0     NaN  NaN
299   0.0     NaN  NaN


In [7]:
# --------------------------------------
# RBF Reconstruction library

# subset - scalar / directional indexes
ix_scalar_subset = [0,1]      # scalar (hs, tp)
ix_directional_subset = [2]   # directional (dir)

# target - scalar / directional indexes
ix_scalar_target = [0,1]      # scalar (Hsig, Tpsmoo, Dir)
ix_directional_target = [2]   # directional (Dir)


# RBF wrappers 
def RBF_Reconstruction_sea(dataset_input):
    
    return RBF_Reconstruction(
    wvs_sea_subset, ix_scalar_subset, ix_directional_subset,
    wvs_sea_target, ix_scalar_target, ix_directional_target,
    dataset_input)

def RBF_Reconstruction_swell(dataset_input):
    
    return RBF_Reconstruction(
    wvs_swl_subset, ix_scalar_subset, ix_directional_subset,
    wvs_swl_target, ix_scalar_target, ix_directional_target,
    dataset_input)

def RBF_Reconstruction_families(data):
   
    # sea
    vs = ['sea_Hs', 'sea_Tp', 'sea_Dir']
    data_sea = data[vs].dropna()
    data_sea.drop(data_sea[(data_sea['sea_Dir'] >= 360)].index, inplace=True)  # fix sea_Dir >> 360 bug    
    rec_sea = RBF_Reconstruction_sea(data_sea.values)
    rec_sea = pd.DataFrame(data=rec_sea, columns=vs, index=data_sea.index)

    # swell 1
    vs = ['swell_1_Hs', 'swell_1_Tp', 'swell_1_Dir']
    data_swl_1 = data[vs].dropna()
    rec_swl_1 = RBF_Reconstruction_swell(data_swl_1.values)
    rec_swl_1 = pd.DataFrame(data=rec_swl_1, columns=vs, index=data_swl_1.index)

    # swell 2
    vs = ['swell_2_Hs', 'swell_2_Tp', 'swell_2_Dir']
    data_swl_2 = data[vs].dropna()
    rec_swl_2 = RBF_Reconstruction_swell(data_swl_2.values)
    rec_swl_2 = pd.DataFrame(data=rec_swl_2, columns=vs, index=data_swl_2.index)
    
    # join nearshore reconstructed data and parse to xarray.Dataset
    rec_waves = pd.concat([rec_sea, rec_swl_1, rec_swl_2], axis=1)

    # return pandas.dataframe
    return rec_waves


In [8]:
# --------------------------------------
# Load simulation waves families

n_sims_DWTs = 10

CE = Climate_Emulator(db_S5.paths.site.EXTREMES.climate_emulator)
# CE.Load()

WVS = CE.LoadSim_All(TCs=True)
WVS

,DWT,sea_Hs,sea_Tp,sea_Dir,swell_1_Hs,swell_1_Tp,swell_1_Dir,swell_2_Hs,swell_2_Tp,swell_2_Dir,wind_speed,wind_dir,mu,tau,ss,n_sim,time
0,19.0,3.140000,9.092714,121.560600,0.755483,10.195459,358.316864,1.426613,16.200777,76.147141,9.392583,85.410378,0.913489,0.041667,0.0,0,2000-01-01 00:00:00
1,26.0,2.044775,9.769193,85.387520,0.976709,12.476293,329.727600,0.369363,11.345125,142.918030,4.648453,182.045914,0.969047,0.680556,0.0,0,2000-01-04 00:00:00
2,18.0,2.542092,9.404038,88.745987,0.764576,11.942608,298.298431,0.285994,13.991345,63.839600,9.380447,73.336884,0.872136,0.750000,0.0,0,2000-01-06 00:00:00
3,33.0,2.217739,10.560315,215.599945,1.089627,10.572375,324.192993,0.334965,11.635722,114.080986,5.940996,229.553375,0.910566,0.041667,0.0,0,2000-01-07 00:00:00
4,32.0,3.612528,8.010935,79.811592,0.811226,12.329054,353.366119,0.473667,10.768278,130.585205,8.783158,81.365089,0.877830,0.041667,0.0,0,2000-01-11 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238056,36.0,2.767216,9.806191,66.129005,0.451771,12.198394,353.091675,1.354785,11.052293,129.878693,7.903176,82.802910,0.920397,1.000000,0.0,9,2999-12-25 00:00:00
1238057,18.0,2.100918,9.149567,238.050858,0.852163,9.857091,35.555481,0.874888,14.374507,153.461426,8.399336,89.851784,0.863095,0.020833,0.0,9,2999-12-26 00:00:00
1238058,26.0,2.547366,9.462438,89.798889,0.903598,16.215944,11.142534,0.155492,19.554379,129.677750,8.476381,54.673023,0.927818,0.833333,0.0,9,2999-12-27 00:00:00
1238059,27.0,2.046063,7.210924,83.981377,0.808038,9.832579,322.732025,0.177390,9.966562,129.394440,8.348703,118.466850,0.942478,0.041667,0.0,9,2999-12-29 00:00:00


In [9]:
# Reconstruct waves by families
WVS_rec = RBF_Reconstruction_families(WVS)

# set indexes values
WVS_rec['n_sim'] = WVS.n_sim
WVS_rec['time'] = WVS.time
WVS_rec['wind_speed'] = WVS.wind_speed
WVS_rec['wind_dir'] = WVS.wind_dir
WVS_rec['mu'] = WVS.mu
WVS_rec['tau'] = WVS.tau
WVS_rec['ss'] = WVS.ss


ix_scalar: 0,  optimization: 25.26 | interpolation: 61.22
ix_scalar: 1,  optimization: 26.54 | interpolation: 59.12
ix_directional: 2,  optimization: 47.47 | interpolation: 117.73
ix_scalar: 0,  optimization: 8.22 | interpolation: 56.13
ix_scalar: 1,  optimization: 24.45 | interpolation: 59.95
ix_directional: 2,  optimization: 26.83 | interpolation: 118.99
ix_scalar: 0,  optimization: 7.99 | interpolation: 56.76
ix_scalar: 1,  optimization: 23.08 | interpolation: 60.06
ix_directional: 2,  optimization: 25.43 | interpolation: 115.46


In [10]:
# store reconstruction
key=os.path.join(db_sim_S5.p_base,'Reconstruction_profile' + str(prf))
WVS_rec.to_pickle(key)
print ('Profile '+ str(prf)+' --> Saved in: ' + key)

Profile 9 --> Saved in: /media/administrador/HD/Dropbox/Guam/teslakit/data/sites/GUAM_CC_S5/HYSWAN/sim/Reconstruction_profile9


In [11]:
#We change the funcion from teslakit.database to include profile number

from teslakit.database import SplitStorage

def Save_SIM_NEARSHORE(self, xds, n_sim, prf):
    ps = self.paths.site.SIMULATION.nearshore

    nm = '{0:08d}'.format(n_sim)  # sim code
    ps_sim = op.join(ps, nm + '_prf_' + str(prf))

    s =  SplitStorage(ps_sim)
    s.Save(xds, safe_time=True)

In [12]:
def waves_hourly(CE, WVS_SIM_rec, n_sim_ce=0, db=[], prf=0):
    '''
    Generate nearshore hourly simulation output. 
    
    - aggregate nearshore waves storms
    - generate hourly hydrographs
    - calculates AWL
    
    Stores simulation nearshore waves at teslakit site database
    '''
    
    # iterate simulations
    for n in range(n_sims_DWTs):
        
        # select simulation
        WVS_SIM_rec_n = WVS_SIM_rec.loc[WVS_SIM_rec['n_sim'] == n]

        # aggregate nearshore variables (Tp from max. energy waves family)
        wvs_agr = Aggregate_WavesFamilies(WVS_SIM_rec_n, a_tp='max_energy')                
        
        # Load TCs storm data from climate emulator
        _, TCS_sim, _ = CE.LoadSim(n_sim = n)    
        TCS_sim = TCS_sim.isel(n_sim = n_sim_ce).copy() 
        TCS_sim = TCS_sim.sel(time=wvs_agr.time)
        
        # calculate intradaily hourly hydrographs for simulated storms
        wvs_h = Intradaily_Hydrograph(wvs_agr, TCS_sim)
        
        # calculate AWL
        wvs_h['AWL'] = AWL(wvs_h.Hs, wvs_h.Tp)
        
        # store hourly simulation offshore data
#         db.Save_SIM_NEARSHORE(wvs_h, n)
        Save_SIM_NEARSHORE(db, wvs_h, n, prf)
                
        print('simulation {0} processed.'.format(n))

# generate intradaily hydrographs


In [13]:
waves_hourly(CE, WVS_rec, db=db_S5, prf=prf)  

simulation 0 processed.
simulation 1 processed.
simulation 2 processed.
simulation 3 processed.
simulation 4 processed.
simulation 5 processed.
simulation 6 processed.
simulation 7 processed.
simulation 8 processed.
simulation 9 processed.
